In [ ]:
from sklearn.datasets import fetch_20newsgroups
from sklearn.decomposition import LatentDirichletAllocation, NMF, TruncatedSVD


In [ ]:
class NewsRecommender:
    """
    обучить систему на корпусе текстов  с помощью тематической модели и метрики, выбранных в результате исследования
    """
    def __init__(self):
        self.model = None
     
    def train(self, texts, n_topics=15):
        # texts = fetch_20newsgroups()
        lda = LatentDirichletAllocation(n_components=n_topics, learning_method="batch")
        lda.fit(texts)
    
    
    """
    выдать k самых пожих новостей для заданного заголовка по функции расстояния, выбранной в результате исследования
    обратите внимание, что text_sample может содержать слова не из обучающего корпуса
    """
    def recommend(self, text_sample, k):
        return ["news_1", "news_2", ... , "news_k"]